In [1]:
!pip install emoji
!pip install transformers

     |████████████████████████████████| 133kB 8.4MB/s 
     |████████████████████████████████| 2.1MB 7.7MB/s 
     |████████████████████████████████| 3.3MB 36.3MB/s 
     |████████████████████████████████| 901kB 53.2MB/s 


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim

gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

print(f'GPU available: {torch.cuda.is_available()}')

Sun May  2 23:10:08 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
import StockSentimentDataset
from torch.utils.data import DataLoader
from sklearn.utils.class_weight import compute_class_weight

# train_ds, val_ds, test_ds, embeddings, featurizer = StockSentimentDataset.load_all_data()

# class_weights = compute_class_weight(class_weight='balanced', classes=[0,1,2], y=train_ds.labels)
# class_weights_tensor = torch.FloatTensor(class_weights)

In [4]:
from transformers import BertModel, BertTokenizer
from dataset_processing import read_twitter_stock_data
import pickle
import re

In [5]:
train_df = read_twitter_stock_data('train.csv')
val_df = read_twitter_stock_data('val.csv')
test_df = read_twitter_stock_data('test.csv')

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
with open('embeddings_limited.pkl', "rb") as f:
        embeddings = pickle.load(f)

In [6]:
def preprocess_text(text):
  text = text.lower()

  url_regex = r'https?://\S+\b|www\.(\w+\.)+\S*'
  user_regex = r'@\w+'
  eyes = "[8:=;]"
  nose = "['`-]?"
  smiley_regex = "({0}{1}[)d]+|[)d]+{1}{0})".format(eyes, nose)
  lol_face_regex = eyes + nose + "p+"
  sad_face_regex = "({0}{1}\\(+|{1}{0}\\))".format(eyes, nose)
  neutral_face_regex = "{0}{1}[\\/|l*]".format(eyes, nose)
  number_regex = r'[-+]?[.\d]*[\d]+[:,.\d]*'
  punct_repitition_regex = r'([!?.]){2,}'
  word_ending_repitition_regex = r'\b(\S*?)(.)\2{2,}\b'

  text = re.sub(r'\s{2,}', ' ', text)
  text = re.sub(r'\'', '', text)
  text = re.sub(url_regex, "<url>", text)
  text = re.sub(user_regex, "<user>", text)
  text = re.sub(smiley_regex, "<smile>", text)
  text = re.sub(lol_face_regex, "<lolface>", text)
  text = re.sub(sad_face_regex, "<sadface>", text)
  text = re.sub(neutral_face_regex, "<neutralface>", text)
  text = re.sub("<3", "<heart>", text)
  text = re.sub(number_regex, "<number>", text)
  text = re.sub(punct_repitition_regex, lambda match: match.group(1) + " <repeat>", text)
  text = re.sub(word_ending_repitition_regex, lambda match: match.group(1) + match.group(2) + " <elong>", text)

  return text

In [ ]:
# def preprocess_data(data):
#   ids = []
#   a_mask = []

#   for d in data:
#     encoded = tokenizer.encode_plus(
#         text = preprocess_text(d),
#         max_length = 64,
#         pad_to_max_length=False,
#         return_attention_mask = True
#     )
#     ids.append(encoded.get('input_ids'))
#     a_mask.append(encoded.get('attention_mask'))
#   # print(ids)
#   ids = torch.tensor(ids)
#   a_mask = torch.tensor(a_mask)
#   return ids, a_mask

In [ ]:
# train_ds, train_masks = preprocess_data(train_df)
# test_ds, test_masks = preprocess_data(test_df)
# val_ds, val_masks = preprocess_data(val_df)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2079: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [6]:
from torch.utils.data import Dataset

class BertSentimentDataset(Dataset):
  def __init__(self, stock_df, tokenizer):
    # self.samples = tokenizer.featurize(stock_df["text"].tolist())
    # lol = tokenizer.batch_encode_plus(
    #     batch_text_or_text_pairs = stock_df['text'],
    #     max_length = 64,
    #     pad_to_max_length=True,
    #     # return_attention_mask = True
    # )
    # self.samples = lol.get('input_ids')
    self.text = stock_df['text'].tolist()
    self.sentiment_id_mapping = {"negative": 0, "neutral": 1, "positive": 2}
    self.labels = [self.sentiment_id_mapping[s] for s in stock_df["sentiment"].tolist()]

  def __len__(self):
    return len(self.text)

  def __getitem__(self, idx):
    if torch.is_tensor(idx):
      idx = idx.tolist()
    return {"text": self.text[idx], "label": self.labels[idx]}

In [26]:
def collate_fn(data):
    # src_ids = [torch.LongTensor(e["sample"]) for e in data]
    src_texts = [e["text"] for e in data]
    lol = tokenizer.batch_encode_plus(
      batch_text_or_text_pairs = src_texts,
      max_length = 100,
      pad_to_max_length=True,
      return_tensors='pt',
      return_attention_mask = True
    )
    labels = [e["label"] for e in data]
    labels_tensor = torch.LongTensor(labels)

    return lol, labels_tensor

In [27]:
train_ds = BertSentimentDataset(train_df, tokenizer)
test_ds = BertSentimentDataset(test_df, tokenizer)
val_ds = BertSentimentDataset(val_df, tokenizer)

In [28]:
class bert_sent(nn.Module):
    def __init__(self, bert, D_out, dropout=0.3):
        
        super().__init__()
        
        self.bert = bert
        
        embedding_dim = bert.config.to_dict()['hidden_size']
        D_in, H = 768, 50

        self.classifier = nn.Sequential(
            nn.Linear(D_in, H),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(H, D_out)
        )
        
        # self.fc = nn.Linear(embedding_dim, output_dim)
        
        # self.dropout = nn.Dropout(dropout)
        
    def forward(self, input_ids, attention_mask):
        # print(input)
        outputs = self.bert(input_ids, attention_mask=attention_mask)
        
        # Extract the last hidden state of the token `[CLS]` for classification task
        # last_hidden_state_cls = outputs[0][:, 0, :]
        last_hidden_state_cls = outputs.pooler_output

        # Feed input to classifier to compute logits
        logits = self.classifier(last_hidden_state_cls)

        return logits
        # text = text.permute(1, 0)
        
        # embedded = self.dropout(self.bert(text)[0])
                
        # embedded = embedded.permute(1, 0, 2)
        
        # predictions = self.fc(self.dropout(embedded))
        
        # return predictions
        # return embedded

In [29]:
bert = BertModel.from_pretrained('bert-base-uncased')

In [11]:
from sklearn.metrics import f1_score
import random

def score_model(model, test_ds, batch_size):
  model.eval()
  with torch.no_grad():
    test_dl = DataLoader(dataset=test_ds, batch_size=batch_size,
      shuffle=False, collate_fn=collate_fn)
    predictions = []
    for x, y in test_dl:
      # x = x.cuda()
      # y = y.cuda()
      # pred_scores = model(x)
      input_ids = x['input_ids'].cuda()
      attention_mask = x['attention_mask'].cuda()
      y = y.cuda()
      # y = torch.nn.functional.one_hot(y, 3)
      pred_scores = model.forward(input_ids, attention_mask)
      # probs = torch.nn.functional.log_softmax(probs, dim=1)

      predictions.extend(torch.argmax(pred_scores, dim=1).tolist())
  return f1_score(test_ds.labels, predictions, labels=[0,1,2], average='micro')

In [34]:
#Training

from random import sample
import tqdm
import os
import subprocess
import random
from transformers import BertModel
from torch.utils.data import DataLoader
from pretrained_embeddings import add_embeddings_to_embedding_matrix

def train(train_ds, model, best_val_model):
  
  #optimizer = optim.SGD(model.parameters(), lr=0.01)
  # optimizer = optim.Adam(model.parameters(), lr=0.001)


  class_parameter_names = ['classifier']
                               
  class_named_params = list(filter(lambda kv: any(key in kv[0] for key in class_parameter_names), model.named_parameters()))
  bert_named_params = list(filter(lambda kv: not any(key in kv[0] for key in class_parameter_names), model.named_parameters()))
  class_params = [e[1] for e in class_named_params]
  bert_params = [e[1] for e in bert_named_params]
  optimizer = optim.AdamW([{'params': class_params}, {'params': bert_params, 'lr': 0.00001 }], lr=0.001)
  
  
  batchSize = 16


  train_dl = DataLoader(dataset=train_ds, batch_size=batchSize, shuffle=True, collate_fn=collate_fn)

  val_dl = DataLoader(dataset=val_ds, batch_size=batchSize, shuffle=False, collate_fn=collate_fn)
  nEpochs = 30
  epoch_stats = []
  best_f1 = 0.0
  best_val_model.load_state_dict(model.state_dict())
  for epoch in range(nEpochs):
    totalLoss = 0.0
    n_samples = 0
    model.train()
    for x, y in tqdm.notebook.tqdm(train_dl, leave=False):
      model.zero_grad()
      input_ids = x['input_ids'].cuda()
      attention_mask = x['attention_mask'].cuda()
      y = y.cuda()
      y = torch.nn.functional.one_hot(y, 3)
      probs = model.forward(input_ids, attention_mask)
      probs = torch.nn.functional.log_softmax(probs, dim=1)
      loss = (torch.neg(probs * y)).sum()
      n_samples+= input_ids.size()[1]

      totalLoss += loss
      loss.backward()
      optimizer.step()
    print(f"loss on epoch {epoch} = {totalLoss}")
    average_train_loss = totalLoss / float(n_samples)

    model.eval()
    total_val_loss = 0.0
    n_val_samples = 0
    with torch.no_grad():
      for x, y in val_dl:
        input_ids = x['input_ids'].cuda()
        attention_mask = x['attention_mask'].cuda()
        y = y.cuda()
        y = torch.nn.functional.one_hot(y, 3)
        probs = model.forward(input_ids, attention_mask)
        probs = torch.nn.functional.log_softmax(probs, dim=1)
        loss = (torch.neg(probs * y)).sum()
        total_val_loss += loss.item()
        n_val_samples+= input_ids.size()[1]
    
    average_val_loss = total_val_loss / float(n_val_samples)
    epoch_stats.append((epoch, average_train_loss, average_val_loss))
    val_f1 = score_model(model, val_ds, batchSize)
    if epoch % 1 == 0:
      print("Epoch {0} finished. Train loss: {1}, Val loss: {2}".format(epoch, average_train_loss, average_val_loss))

    if val_f1 > best_f1:
      best_f1 = val_f1
      n_epochs_without_improvement = 0
      best_val_model.load_state_dict(model.state_dict())
    # else:
    #   n_epochs_without_improvement += 1
    #   if n_epochs_without_improvement == 10:
    #     break

  return best_f1, epoch_stats

model = bert_sent(bert, 3).cuda()
best_val_model = bert_sent(bert, 3).cuda()
train(train_ds, model, best_val_model)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2079: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


loss on epoch 0 = 74.7717056274414
Epoch 0 finished. Train loss: 0.013117843307554722, Val loss: 0.3281387387789213


loss on epoch 1 = 10.126446723937988
Epoch 1 finished. Train loss: 0.0017765696393325925, Val loss: 0.3321602960733267


loss on epoch 2 = 2.403146743774414
Epoch 2 finished. Train loss: 0.0004216046945657581, Val loss: 0.36445328639103813


loss on epoch 3 = 2.840212821960449
Epoch 3 finished. Train loss: 0.0004982829559594393, Val loss: 0.4152742393200214


...

loss on epoch 26 = 39.181488037109375
Epoch 26 finished. Train loss: 0.006873945705592632, Val loss: 0.35200862884521483


loss on epoch 27 = 12.621590614318848
Epoch 27 finished. Train loss: 0.0022143141832202673, Val loss: 0.4222580572275015


loss on epoch 28 = 20.051591873168945
Epoch 28 finished. Train loss: 0.0035178232938051224, Val loss: 0.37302370071411134


loss on epoch 29 = 6.702250957489014
Epoch 29 finished. Train loss: 0.00117583351675421, Val loss: 0.4236920268719013


(0.7295918367346939,
 [(0,
   tensor(0.0131, device='cuda:0', grad_fn=<DivBackward0>),
   0.3281387387789213),
  (1,
   tensor(0.0018, device='cuda:0', grad_fn=<DivBackward0>),
   0.3321602960733267),
  (2,
   tensor(0.0004, device='cuda:0', grad_fn=<DivBackward0>),
   0.36445328639103813),
  (3,
   tensor(0.0005, device='cuda:0', grad_fn=<DivBackward0>),
   0.4152742393200214),
  (4,
   tensor(0.0007, device='cuda:0', grad_fn=<DivBackward0>),
   0.4253117715395414),
  (5,
   tensor(9.7618e-05, device='cuda:0', grad_fn=<DivBackward0>),
   0.4395380188868596),
  (6,
   tensor(7.9952e-05, device='cuda:0', grad_fn=<DivBackward0>),
   0.4462832788320688),
  (7,
   tensor(0.0001, device='cuda:0', grad_fn=<DivBackward0>),
   0.47072947061978854),
  (8,
   tensor(1.5221e-05, device='cuda:0', grad_fn=<DivBackward0>),
   0.47762693771949183),
  (9,
   tensor(2.0568e-05, device='cuda:0', grad_fn=<DivBackward0>),
   0.4878116548978365),
  (10,
   tensor(1.6204e-05, device='cuda:0', grad_fn=<DivBa

In [35]:
batchSize = 16

In [36]:
print(score_model(model, train_ds, batchSize))
print(score_model(model, test_ds, batchSize))
print(score_model(model, val_ds, batchSize))

print(score_model(best_val_model, train_ds, batchSize))
print(score_model(best_val_model, test_ds, batchSize))
print(score_model(best_val_model, val_ds, batchSize))

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2079: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


0.9988998899889989
0.6820512820512821
0.6887755102040817
0.9988998899889989
0.6717948717948717
0.6887755102040817
